# 데이터 수집

## 제품 데이터 수집

### excel -> csv

In [ ]:
%pip install -q ipykernel

In [ ]:
%pip install -q pandas

In [ ]:
%pip install -q openpyxl

In [ ]:
import pandas as pd

# 엑셀 파일의 첫 번째 시트를 읽어와 DataFrame으로 만듭니다.
# 만약 특정 시트를 변환하고 싶다면, sheet_name='시트명'을 추가하세요.
df = pd.read_excel('data.xlsx')

# DataFrame을 CSV 파일로 저장합니다.
# index=False는 행 번호를 CSV에 저장하지 않도록 합니다.
df.to_csv('product_data_raw.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

# 1. 제품 데이터 불러오기
product_data = pd.read_csv('product_data_raw.csv')

# 2. '제품명' 컬럼을 기준으로 중복된 행 확인
# 'keep=False'를 사용해 중복된 모든 행을 표시합니다.
duplicate_products = product_data[product_data.duplicated(subset=['제품명'], keep=False)]

# 3. 중복된 제품명 출력
if not duplicate_products.empty:
    print("--- [중복된 제품명 목록] ---")
    print(duplicate_products.sort_values(by='제품명'))
    print("-" * 30)
    print(f"총 {len(duplicate_products['제품명'].unique())}개의 제품명이 중복되었습니다.")
    print(f"총 {len(duplicate_products)}개의 중복된 행이 발견되었습니다.")
else:
    print("중복된 제품명이 없습니다.")

### `전성분` 컬럼 표시 형식 통일
- ;이나 ,로 구분되어 있던 성분명을 ;로 구분되도록 표시 형식 통일
- `1,2-헥산다이올, (15,000ppm)`과 같은 예외 형식들은 직접 찾아 표시 형식 변환

In [ ]:
import pandas as pd
import csv

# 1. CSV 파일을 불러옵니다.
df = pd.read_csv('product_data_raw.csv')

# 2. 첫 번째 열(인덱스 열)을 삭제합니다.
df = df.drop(df.columns[0], axis=1)

# 3. '전성분' 컬럼의 구분 기호를 통일합니다.
df['전성분'] = df['전성분'].str.replace(',', ';', regex=True)
df['전성분'] = df['전성분'].str.replace(' ', '', regex=True)

# 4. 정제된 DataFrame을 새로운 CSV 파일로 저장합니다.
df.to_csv('product_data_cleaned.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8-sig')

### 고유 성분명 리스트 추출

In [ ]:
import pandas as pd

# 1. CSV 파일을 불러옵니다.
df = pd.read_csv('product_data_cleaned.csv')

# 2. '전성분' 컬럼의 문자열을 세미콜론(;)을 기준으로 분리하여 리스트로 만듭니다.
all_ingredients = df['전성분'].str.split(';')

# 3. 모든 성분 리스트를 하나로 합치고 중복을 제거합니다.
unique_ingredients = set()
for sublist in all_ingredients:
    for item in sublist:
        # 공백을 제거하고 빈 문자열이 아닌 경우에만 추가
        cleaned_item = item.strip()
        if cleaned_item:
            unique_ingredients.add(cleaned_item)

# 4. 중복이 제거된 성분 목록을 다시 리스트로 변환합니다.
unique_ingredients_list = sorted(list(unique_ingredients))

print("--- 고유 성분명 목록 추출 완료 ---")
print(f"총 {len(unique_ingredients_list)}개의 고유 성분명이 발견되었습니다.")
print("\n고유 성분명 리스트 :")
print(unique_ingredients_list)

**빈도수 5회 이상인 성분 개수 추출**
- 1015개 중 208개

In [ ]:
import pandas as pd
from collections import Counter

# 1. product_data_cleaned.csv 파일의 전체 데이터 읽기
df = pd.read_csv('product_data_cleaned.csv')

# 2. 모든 성분 추출 및 빈도 계산
all_ingredients = []
# 세미콜론(;)을 기준으로 성분명을 분리합니다.
for ingredients_str in df['전성분']:
    ingredients_list = [ing.strip() for ing in ingredients_str.split(';')]
    all_ingredients.extend(ingredients_list)

# 3. 빈도수 계산 및 필터링
ingredient_counts = Counter(all_ingredients)
# 매핑 작업을 위해 남길 최소 빈도수를 설정합니다.
frequency_threshold = 5
filtered_ingredients = {ingredient for ingredient, count in ingredient_counts.items() if count >= frequency_threshold}

# 4. 결과 출력
print(f"--- [성분 빈도수 분석 결과] ---")
print(f"총 고유 성분 수: {len(ingredient_counts)}개")
print(f"빈도수 {frequency_threshold}회 미만인 성분 수: {len(ingredient_counts) - len(filtered_ingredients)}개")
print(f"빈도수 {frequency_threshold}회 이상인 성분 수: {len(filtered_ingredients)}개")
print("-" * 30)

# 매핑 작업 대상이 될 상위 N개의 성분 목록만 출력하여 확인합니다.
print(f"\n--- [빈도수 {frequency_threshold}회 이상인 성분 목록 (일부)] ---")
top_n = 20  # 출력할 상위 개수 설정
sorted_ingredients = sorted(filtered_ingredients, key=lambda x: ingredient_counts[x], reverse=True)

for i, ingredient in enumerate(sorted_ingredients):
    if i >= top_n:
        print("...")
        break
    print(f" - {ingredient} (빈도수: {ingredient_counts[ingredient]}회)")

In [ ]:
import pandas as pd
from collections import Counter

# 1. product_data_cleaned.csv 파일에서 '전성분' 컬럼만 읽기
df = pd.read_csv('product_data_cleaned.csv', usecols=['전성분'])

# 2. 모든 성분 추출 및 빈도 계산
all_ingredients = []
for ingredients_str in df['전성분']:
    ingredients_list = [ing.strip() for ing in ingredients_str.split(';')]
    all_ingredients.extend(ingredients_list)

ingredient_counts = Counter(all_ingredients)
frequency_threshold = 5
# 필터링된 성분 목록을 리스트로 저장합니다.
filtered_ingredients = [ingredient for ingredient, count in ingredient_counts.items() if count >= frequency_threshold]

# 3. 매핑 딕셔너리(사전) 만들기
# 아래 예시처럼 filtered_ingredients 리스트를 참고하여 직접 매핑 사전을 구축합니다.
# 이 딕셔너리를 완성하는 것이 이 단계의 목표입니다.
mapping_dict = {
    "정제수": "Aqua",
    "글리세린": "Glycerin",
    "징크옥사이드": "Zinc Oxide",
    # ... 여기에 84개 성분에 대한 매핑 정보를 추가합니다.
}

# 작업에 필요한 필터링된 성분 목록을 출력합니다.
print("--- [ICNI 매핑 사전 구축 대상 성분 목록] ---")
for i, ingredient in enumerate(sorted(filtered_ingredients), 1):
    print(f"{i}. {ingredient}")

## 성분 데이터 수집
- ICNI_mapping.csv : 한국어성분명을 가지고 대한화장품협회를 통해 영문표준명 수집 완료.
- ewg_ratings.csv : 영문표준명을 가지고 EWG 등급 수집 완료. (없는 값도 드물게 존재)
- ingredient_data.csv : 위에 두 csv 파일을 '영문표준명' 컬럼 기준으로 병합.

In [ ]:
import pandas as pd

# 1. ICNI 매핑 사전 파일 불러오기
# 이 파일은 "한국어성분명", "영문표준명" 두 컬럼을 가지고 있습니다.
icni_df = pd.read_csv('ICNI_mapping.csv')

# 2. 여러분이 만든 EWG 등급 정보 파일 불러오기
# 이 파일은 "영문표준명", "EWG등급" 두 컬럼을 가지고 있습니다.
ewg_df = pd.read_csv('ewg_ratings.csv')

# 3. 두 데이터프레임 합치기
# '영문표준명' 컬럼을 기준으로 병합합니다.
# how='left'를 사용해 매핑 사전의 모든 성분을 기준으로 합칩니다.
ingredient_data = pd.merge(icni_df, ewg_df, on='영문표준명', how='left')

# 4. 결과 출력
print("--- [통합된 성분 데이터] ---")
print(ingredient_data)

# 5. 최종 데이터프레임 CSV 파일로 저장
# 나중에 필요할 때 사용할 수 있도록 파일로 저장해 둡니다.
ingredient_data.to_csv('ingredient_data.csv', index=False, encoding='utf-8-sig')

### EWG 점수 계산
- 목적 : 단순히 성분들의 등급을 나열하는 것을 넘어 제품 전체의 안전성을 객관적인 수치로 평가하기 위함.
- **제품 안전성 평가**: EWG 등급이 낮은 성분이 많이 들어갈수록 제품의 평균 점수는 낮아짐.
    - 직관적으로 제품의 안전성을 알려줄 수 있음.
- **제품 순위 매기기**: 모든 제품의 평균 EWG 점수를 계산하면, 점수가 낮은 순서대로 제품을 정렬하여 가장 안전한 제품부터 추천 가능.
    - 신뢰를 줄 수 있는 중요한 추천 기준이 됨.

In [ ]:
import pandas as pd
import numpy as np

# 1. EWG 등급이 포함된 최종 성분 데이터 불러오기
ingredient_data = pd.read_csv('ingredient_data.csv')

# 2. 제품 데이터 불러오기
product_data = pd.read_csv('product_data_cleaned.csv')

# 3. 각 성분의 EWG 등급을 찾아주는 딕셔너리 생성
# '한국어성분명'을 키로, 'EWG등급'을 값으로 하는 딕셔너리
ewg_ratings_dict = ingredient_data.set_index('한국어성분명')['EWG등급'].to_dict()

# 4. 제품별 유해성 점수 계산 함수 정의
def calculate_product_score(ingredients_str):
    if pd.isna(ingredients_str):
        return np.nan
        
    ingredient_list = [ing.strip() for ing in ingredients_str.split(';')]
    scores = []
    for ingredient in ingredient_list:
        score = ewg_ratings_dict.get(ingredient, np.nan)
        if pd.notna(score):
            scores.append(score)
    
    if scores:
        return np.mean(scores)
    else:
        return np.nan

# 5. 제품별 유해성 점수 컬럼 추가
product_data['유해성_점수'] = product_data['전성분'].apply(calculate_product_score)

# 6. 결과 출력
print("--- [제품별 유해성 점수 (상위 10개)] ---")
print(product_data.sort_values(by='유해성_점수', ascending=True).head(10))

# 7. 최종 파일로 저장
product_data.to_csv('product_with_score.csv', index=False, encoding='utf-8-sig')

### test_1.csv
- 피부 타입 : 지성, 피부 고민 : 여드름
- `피부 타입: 지성`에 따라 주의 성분이 제품에 포함되어 있으면 감점을 주고 `피부 고민: 여드름`에 따라 효능 성분이 제품에 포함되어 있으면 가산점을 주며 최종 점수 계산 후 정렬시켜 파일에 저장함.

In [ ]:
%pip install -q tabulate

In [ ]:
import pandas as pd
import numpy as np

# 1. 데이터 불러오기
# 여러분의 파일명을 그대로 사용합니다.
product_data = pd.read_csv('product_data_cleaned.csv')
ingredient_data = pd.read_csv('ingredient_data.csv')

# 2. EWG 등급 딕셔너리 생성 (한국어 성분명 기준)
ewg_ratings_dict = ingredient_data.set_index('한국어성분명')['EWG등급'].to_dict()

# 3. 피부 타입별 주의 성분 및 효능 성분 사전 정의
oily_skin_caution_ingredients = ["스테아릭애씨드", "팔미틱애씨드", "미리스틱애씨드", "시어버터", "코코넛 오일"]
dry_skin_caution_ingredients = ["변성 알코올", "에탄올"]
sensitive_skin_caution_ingredients = ["향료", "리날룰", "리모넨", "카프릴로일살리실릭애씨드"]
atopy_skin_caution_ingredients = ["향료", "리날룰", "리모넨", "라우릭애씨드", "미리스틱애씨드", "스테아릭애씨드", "팔미틱애씨드"]

acne_efficacy_ingredients = ["병풀추출물", "나이아신아마이드", "카프릴로일살리실릭애씨드"]
brightening_efficacy_ingredients = ["나이아신아마이드", "아스코빅애씨드", "울금뿌리추출물"]

# 4. 최종 점수 계산 함수 정의
def calculate_final_score(ingredients_str, user_skin_type, user_concern):
    if pd.isna(ingredients_str):
        return np.nan
        
    ingredient_list = [ing.strip() for ing in ingredients_str.split(';')]
    
    ewg_scores = []
    for ingredient in ingredient_list:
        score = ewg_ratings_dict.get(ingredient)
        if pd.notna(score):
            ewg_scores.append(score)
    ewg_avg_score = np.nan if not ewg_scores else np.mean(ewg_scores)
    
    efficacy_score = 0
    caution_penalty = 0

    if user_concern == '여드름':
        for ingredient in ingredient_list:
            if ingredient in acne_efficacy_ingredients:
                efficacy_score -= 1
    elif user_concern == '미백':
        for ingredient in ingredient_list:
            if ingredient in brightening_efficacy_ingredients:
                efficacy_score -= 1
    
    if user_skin_type == '지성':
        for ingredient in ingredient_list:
            if ingredient in oily_skin_caution_ingredients:
                caution_penalty += 3
    elif user_skin_type == '민감성':
        for ingredient in ingredient_list:
            if ingredient in sensitive_skin_caution_ingredients:
                caution_penalty += 3
    
    final_score = ewg_avg_score + efficacy_score + caution_penalty
    return final_score

# 5. '지성 피부'와 '여드름' 고민을 가진 사용자를 가정하고 점수 계산
user_skin_type = '지성'
user_concern = '여드름'
product_data['최종_점수'] = product_data['전성분'].apply(
    lambda x: calculate_final_score(x, user_skin_type, user_concern)
)

# 6. 소수점 정리 및 데이터프레임 정렬
product_data['최종_점수'] = product_data['최종_점수'].round(3)
sorted_product_data = product_data.sort_values(by='최종_점수', ascending=True)

# 7. 결과 출력 및 파일 저장
print(f"--- [고객의 피부 타입: {user_skin_type}, 고민: {user_concern}에 대한 추천 순위] ---")
print(sorted_product_data[['제품명', '브랜드명', '최종_점수']].head(10).to_markdown(index=False, numalign="left", stralign="left"))

sorted_product_data.to_csv('test_1.csv', index=False, encoding='utf-8-sig')